# Scraping many pages + Using Selenium

## The pages we'll be looking at

If I wanted to read specific information about a specfic mine, it takes a few steps. **Do these steps with your browser before you try any programming.**

1. Visit the [Mine Data Retrieval System](https://arlweb.msha.gov/drs/drshome.htm)
2. Scroll down to **Mine Identification Number (ID) Search**
3. Type in a mine ID number, such as `3503598`, click **Search**
4. I'm on a page! It lists the MINE NAME and MINE OWNER.

After searching for and finding a mine, I can use this page to **find reports about this mine**. Some of the reports are on accidents, violations, inspections, health samples and more. To get those reports:

1. Search for a mine (if you haven't already)
2. Scroll down and change **Beginning Date** to `1/1/1995` (violation reports begin in 1995, accidents begin in 1983)
3. Select the report type of `Violations`
4. Click **Get Report**
5. I'm on a page! It lists ALL OF THE MINE'S VIOLATIONS.

By changing the report type you're searching for you can find all sorts of different data.

# Researching mine information

## Preparation 

### When you search for information on a specific mine, what URL should Selenium visit first?

- *TIP: the answer is NOT `https://arlweb.msha.gov/drs/ASP/BasicMineInfonew.asp`*

In [1]:
from selenium import webdriver
driver = webdriver.Chrome()
driver.get("https://arlweb.msha.gov/drs/drshome.htm")

### How can you identify the text field we're going to type the Mine ID into?

Selenium can find elements by:

- name
- Class
- ID
- CSS selector (**ASK ME WHAT THIS IS** if you don't know)
- XPath (**ASK ME WHAT THIS IS** because you definitely don't know)
- Link text
- Partial link text

So in other words, what's unique about this element?

In [15]:
Mine_ID = driver.find_elements_by_xpath('//*[@id="inputdrs"]')
Mine_ID[1].send_keys("3901432")

### How can you identify the search button we're going to click, or the form we're going to submit?

Selenium can submit forms by either

- Selecting the form and using `.submit()`, or
- Selecting the button and using `.click()`

You only need to be able to get **one, not both.**

In [16]:
search = driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input')
search.click()


### Use Selenium to search using the mine ID `3901432`. Get me the operator's name by scraping.

- *TIP: You can find elements/text using Selenium, or use BeautifulSoup with `doc = BeautifulSoup(driver.page_source)`*

In [19]:
from bs4 import BeautifulSoup
doc = BeautifulSoup(driver.page_source, "html.parser")
doc.find_all("b")[5].text

'Krueger Brothers Gravel & Dirt'

# Using .apply to find data about SEVERAL mines

The file `mines-subset.csv` has a list of mine IDs. We're going to scrape the operator's name for each of those mines.

### Open up `mines-subset.csv` and save it into a dataframe

In [25]:
import pandas as pd

df_mines = pd.read_csv("mines-subset.csv")
df_mines["id"] = df_mines["id"].astype(str)
df_mines.head()



,id
0,4104757
1,801306
2,3609931


### Open up `mines-subset.csv` in a text editor, then look at your dataframe. Is something different about them?

### Scrape the operator's name for each of those mines and print it

- *TIP: use .apply and a function*
- *TIP: If you need help with .apply, look at the "Using apply in pandas" notebook *

In [55]:
def mine_ids(row):
    driver.get("https://arlweb.msha.gov/drs/drshome.htm")
    Mine_ID = driver.find_elements_by_xpath('//*[@id="inputdrs"]')[1]
    Mine_ID.send_keys(row['id'])
    search = driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input')
    search.click()
    doc = BeautifulSoup(driver.page_source, "html.parser")
    bs = doc.find_all("b")
    if len(bs) > 5:
        mine_names = bs[5].text
        return pd.Series({
            "Mine ids": mine_names
        })

df_mines.apply(mine_ids, axis=1)




,Mine ids
0,Dirt Works
1,None
2,M.R. Dirt Inc.


In [60]:
df_mines_merged = df_mines.apply(mine_ids, axis=1).join(df_mines)



In [61]:
df_mines_merged

,Mine ids,id
0,Dirt Works,4104757
1,None,801306
2,M.R. Dirt Inc.,3609931


### Scrape the operator's name and save it into a new column

- *TIP: Use .apply and a function*
- *TIP: Remember to use `return`*

In [57]:
df_mines_merged = df_mines.apply(mine_ids, axis=1).join(df_mines)
df_mines_merged

<function __main__.mine_ids>

# Researching mine violations

Read the very top again to remember how to find mine violations

### When you search for a mine's violations, what URL is Selenium going to start on?

- *TIP: `requests` can send form data to load in the middle of a bunch of steps, but Selenium has to start at the beginning

In [ ]:
Mine_ID = driver.find_elements_by_xpath('//*[@id="inputdrs"]')
Mine_ID[1].send_keys("3901432")

### When you're searching for violations from the Mine Information page, how are you going to identify the "Beginning Date" field?

In [6]:
begin_date = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[2]/tbody/tr[2]/td/font/input[1]')
begin_date.send_keys("1/1/1995")

### When you're searching for violations from the Mine Information page, how are you going to identify the "Violations" button?

In [8]:
violaitons = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[3]/tbody/tr[2]/td[2]/table/tbody/tr[1]/td/input')
violaitons.click()

### When you're searching for violations from the Mine Information page, how are you going to identify the form or the button to click to get a list of the violations?

In [9]:
get_report = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[3]/tbody/tr[3]/td[2]/input')
get_report.click()

### Using the mine ID `3901432`, scrape all of their violations since 1/1/1995

**Save this into a CSV called `3901432-violations.csv`.** This CSV must include the following fields:

- Citation number
- Case number
- Standard violated
- Link to standard
- Proposed penalty
- Amount paid to date

**Tips:**

- *TIP: It's probably worth it to print them all first, then save them to a CSV once you know it's all working.*
- *TIP: You'll use the parent pattern - get the ROWS first (tr), then loop through and get the TABLE CELLS (td)*

In [10]:
from bs4 import BeautifulSoup
doc = BeautifulSoup(driver.page_source, "html.parser")

In [93]:
mines = []
for i in doc.find_all("tr")[28:-1]:
    dictionary = {}
    citation_no = i.find_all("font")[4]
    if citation_no:
        dictionary["Citation number"] = citation_no.text.strip()
        #print(citation_no.text)
    case_num = i.find_all("font")[6]
    if case_num:
        dictionary["Case number"] = case_num.text.strip()
        #print(case_num.text)
    standard_vio = i.find_all("font")[21]
    if standard_vio:
        dictionary["Standard violated"] = standard_vio.text.strip()
        #print(standard_vio.text)
    standard_link = i.find_all("font")[20]
    standard_link_1 = standard_link("a")
    if standard_link_1:
        for p in standard_link_1:
            standard_link_2 = p["href"]
            dictionary["Link to standard"] = standard_link_2
    prop_penalty = i.find_all("font")[23]
    if prop_penalty:
        dictionary["Proposed penalty"] = prop_penalty.text.strip()
        #print(prop_penalty.text)
    A_P_T_D = i.find_all("font")[24]
    if A_P_T_D:
        dictionary["Amount paid to date"] = A_P_T_D.text.strip()
        #print(A_P_T_D.text)
    mines.append(dictionary)

In [94]:
mines

[{'Amount paid to date': '100.00',
  'Case number': '000361866',
  'Citation number': '8750964',
  'Link to standard': 'http://www.gpo.gov/fdsys/pkg/CFR-2014-title30-vol1/pdf/CFR-2014-title30-vol1-sec56-18010.pdf',
  'Proposed penalty': '100.00',
  'Standard violated': '56.18010'},
 {'Amount paid to date': '100.00',
  'Case number': '000260865',
  'Citation number': '6426439',
  'Link to standard': 'http://www.gpo.gov/fdsys/pkg/CFR-2011-title30-vol1/pdf/CFR-2011-title30-vol1-sec56-4201.pdf',
  'Proposed penalty': '100.00',
  'Standard violated': '56.4201(a)(2)'},
 {'Amount paid to date': '100.00',
  'Case number': '000260865',
  'Citation number': '6426438',
  'Link to standard': 'http://www.gpo.gov/fdsys/pkg/CFR-2011-title30-vol1/pdf/CFR-2011-title30-vol1-sec56-4101.pdf',
  'Proposed penalty': '100.00',
  'Standard violated': '56.4101'},
 {'Amount paid to date': '100.00',
  'Case number': '000260865',
  'Citation number': '6588189',
  'Link to standard': 'http://www.gpo.gov/fdsys/pkg/

In [95]:
import pandas as pd

df = pd.DataFrame(mines)

df.to_csv("3901432-violations.csv", index = False)
df_mines = pd.read_csv("3901432-violations.csv")


In [96]:
df.head()

,Amount paid to date,Case number,Citation number,Link to standard,Proposed penalty,Standard violated
0,100.00,000361866,8750964,http://www.gpo.gov/fdsys/pkg/CFR-2014-title30-...,100.00,56.18010
1,100.00,000260865,6426439,http://www.gpo.gov/fdsys/pkg/CFR-2011-title30-...,100.00,56.4201(a)(2)
2,100.00,000260865,6426438,http://www.gpo.gov/fdsys/pkg/CFR-2011-title30-...,100.00,56.4101
3,100.00,000260865,6588189,http://www.gpo.gov/fdsys/pkg/CFR-2011-title30-...,100.00,56.14200
4,100.00,000238554,6588210,http://www.gpo.gov/fdsys/pkg/CFR-2010-title30-...,100.00,50.30(a)


# Using .apply to save mine data for SEVERAL mines

The file `mines-subset.csv` has a list of mine IDs. We're going to scrape the operator's name for each of those mines.

### Open up `mines-subset.csv` and save it into a dataframe

In [102]:
df_mines_ids = pd.read_csv("mines-subset.csv", dtype={"id": "str"})
df_mines_ids.head()


,id
0,4104757
1,0801306
2,3609931


### Scrape the violations for each mine

**Save each mine's violations into separate CSV files.** Each CSV file must include the following fields:

- Citation number
- Case number
- Standard violated
- Link to standard
- Proposed penalty
- Amount paid to date

Make sure you are saving them into **separate files.** It might be nice to name them after the mine id.

- *TIP: Use .apply for this*
- *TIP: Print out the ID before you start scraping. That way you can take that ID and search manually to see if there is anything weird about the results.*
- *TIP: If you need help with .apply, look at the "Using apply in pandas" notebook 
- *TIP: It's probably worth it to print the fields first, then save them to a CSV once you know it's all working.*

In [112]:
def mine_ids_steroids(row):
    driver.get("https://arlweb.msha.gov/drs/drshome.htm")
    Mine_ID = driver.find_elements_by_xpath('//*[@id="inputdrs"]')[1]
    Mine_ID.send_keys(row['id'])
    search = driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input')
    search.click()
    begin_date = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[2]/tbody/tr[2]/td/font/input[1]')
    begin_date.send_keys("1/1/1995")
    violaitons = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[3]/tbody/tr[2]/td[2]/table/tbody/tr[1]/td/input')
    violaitons.click()
    get_report = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[3]/tbody/tr[3]/td[2]/input')
    get_report.click()
    doc = BeautifulSoup(driver.page_source, "html.parser")
    mines = []
    for i in doc.find_all("tr")[28:-1]:
        try:
            dictionary = {}
            citation_no = i.find_all("font")[4]
            if citation_no:
                dictionary["Citation number"] = citation_no.text.strip()
                #print(citation_no.text)
            case_num = i.find_all("font")[6]
            if case_num:
                dictionary["Case number"] = case_num.text.strip()
                #print(case_num.text)
            standard_vio = i.find_all("font")[21]
            if standard_vio:
                dictionary["Standard violated"] = standard_vio.text.strip()
                #print(standard_vio.text)
            standard_link = i.find_all("font")[20]
            standard_link_1 = standard_link("a")
            if standard_link_1:
                for p in standard_link_1:
                    standard_link_2 = p["href"]
                    dictionary["Link to standard"] = standard_link_2
            prop_penalty = i.find_all("font")[23]
            if prop_penalty:
                dictionary["Proposed penalty"] = prop_penalty.text.strip()
                #print(prop_penalty.text)
            A_P_T_D = i.find_all("font")[24]
            if A_P_T_D:
                dictionary["Amount paid to date"] = A_P_T_D.text.strip()
                #print(A_P_T_D.text)
            mines.append(dictionary)
        except:
            print("Failed on mine id", row['id'])
            pass
        
    df = pd.DataFrame(mines)
    df.to_csv(row["id"] + ".csv", index = False)

df_mines_ids.apply(mine_ids_steroids, axis=1)

Failed on mine id 0801306
Failed on mine id 0801306
Failed on mine id 3609931


0    None
1    None
2    None
dtype: object

In [113]:
df_mine_4104757 = pd.read_csv("4104757.csv")
df_mine_4104757.head()

,Amount paid to date,Case number,Citation number,Link to standard,Proposed penalty,Standard violated
0,100.0,374480,8778046,http://www.gpo.gov/fdsys/pkg/CFR-2014-title30-...,100.0,56.14132(a)
1,162.0,374480,8778047,http://www.gpo.gov/fdsys/pkg/CFR-2014-title30-...,162.0,56.18010
2,243.0,345454,8771783,http://www.gpo.gov/fdsys/pkg/CFR-2014-title30-...,243.0,56.9300(a)
3,100.0,345454,8771784,http://www.gpo.gov/fdsys/pkg/CFR-2014-title30-...,100.0,50.30(a)
4,100.0,348280,8771781,http://www.gpo.gov/fdsys/pkg/CFR-2014-title30-...,100.0,56.14100(b)


In [114]:
df_mine_0801306 = pd.read_csv("0801306.csv")
df_mine_0801306.head()

,Amount paid to date,Case number,Citation number,Link to standard,Proposed penalty,Standard violated
0,351.00,427623,8638781,http://www.gpo.gov/fdsys/pkg/CFR-2016-title30-...,351.00,56.12028
1,100.00,411633,8903432,http://www.gpo.gov/fdsys/pkg/CFR-2016-title30-...,100.00,47.41(a)
2,100.00,411633,8903433,http://www.gpo.gov/fdsys/pkg/CFR-2016-title30-...,100.00,56.14100(b)
3,117.00,411633,8903435,http://www.gpo.gov/fdsys/pkg/CFR-2016-title30-...,117.00,56.9300(a)
4,117.00,411633,8903434,http://www.gpo.gov/fdsys/pkg/CFR-2016-title30-...,117.00,46.11(d)


In [115]:
df_mine_3609931 = pd.read_csv("3609931.csv")
df_mine_3609931.head()

,Amount paid to date,Case number,Citation number,Link to standard,Proposed penalty,Standard violated
0,114.0,421654,8928850,http://www.gpo.gov/fdsys/pkg/CFR-2016-title30-...,114.0,56.9301
1,100.0,380669,8807882,http://www.gpo.gov/fdsys/pkg/CFR-2015-title30-...,100.0,56.14132(a)
2,100.0,282555,8650963,http://www.gpo.gov/fdsys/pkg/CFR-2012-title30-...,100.0,56.14100(b)
3,100.0,274355,8650926,http://www.gpo.gov/fdsys/pkg/CFR-2011-title30-...,100.0,56.1000
4,100.0,262818,8645896,http://www.gpo.gov/fdsys/pkg/CFR-2011-title30-...,100.0,56.14107(a)
